## Megamarket

#### Transacciones PDF 

In [182]:
pip install pymupdf pdfplumber tabula-py camelot-py[cv] pytesseract Pillow pandas camelot

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [183]:
# Lectura de pdf
import fitz  
import pdfplumber
import tabula
import pytesseract
from PIL import Image
import pandas as pd
import io
import re
import pandas as pd

In [184]:
#funcion de limpieza de columnas : 

def limpiar_columnas(df):
    # Convertimos los nombres de columna a string
    df.columns = df.columns.astype(str)

    # Normalizamos las tildes usando unicodedata
    df.columns = [
        unicodedata.normalize('NFKD', col)
        .encode('ascii', 'ignore')
        .decode('utf-8')
        for col in df.columns
    ]

    # Aplicamos las transformaciones
    df.columns = (
        pd.Index(df.columns)
        .str.strip()
        .str.lower()
        .str.replace(' ', '_')
        .str.replace(r'[^\w\s]', '', regex=True)
    )
    
    return df


In [185]:


class PDFExtractorAdaptativo:
    def __init__(self):
        self.metodos_texto = ['pdfplumber', 'pymupdf', 'ocr']
        self.metodos_tablas = ['tabula', 'pdfplumber']  # Quitamos 'camelot'
    
    def detectar_tipo_contenido(self, archivo_pdf):
        """Detecta qué tipo de contenido tiene el PDF"""
        try:
            doc = fitz.open(archivo_pdf)
            primera_pagina = doc[0]
            
            texto = primera_pagina.get_text().strip()
            imagenes = primera_pagina.get_images()
            
            with pdfplumber.open(archivo_pdf) as pdf:
                page = pdf.pages[0]
                tablas_detectadas = page.find_tables()
            
            resultado = {
                'tiene_texto': len(texto) > 50,
                'tiene_imagenes': len(imagenes) > 0,
                'posibles_tablas': len(tablas_detectadas) > 0,
                'es_escaneado': len(texto) < 50 and len(imagenes) > 0,
                'calidad_texto': 'alta' if len(texto) > 200 else 'baja'
            }
            
            doc.close()
            return resultado
            
        except Exception as e:
            return {'error': str(e)}
    
    def extraer_texto_robusto(self, archivo_pdf):
        """Extrae texto probando diferentes métodos"""
        metodos_resultados = {}
        
        # pdfplumber
        try:
            with pdfplumber.open(archivo_pdf) as pdf:
                texto = ""
                for page in pdf.pages:
                    texto += page.extract_text() + "\n"
                metodos_resultados['pdfplumber'] = texto
        except Exception as e:
            metodos_resultados['pdfplumber'] = f"Error: {e}"
        
        # pymupdf
        try:
            doc = fitz.open(archivo_pdf)
            texto = ""
            for page in doc:
                texto += page.get_text() + "\n"
            doc.close()
            metodos_resultados['pymupdf'] = texto
        except Exception as e:
            metodos_resultados['pymupdf'] = f"Error: {e}"
        
        # OCR
        try:
            doc = fitz.open(archivo_pdf)
            texto = ""
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                pix = page.get_pixmap()
                img_data = pix.tobytes("png")
                img = Image.open(io.BytesIO(img_data))
                texto += pytesseract.image_to_string(img, lang='spa') + "\n"
            doc.close()
            metodos_resultados['ocr'] = texto
        except Exception as e:
            metodos_resultados['ocr'] = f"Error: {e}"
        
        # Seleccionar mejor resultado
        mejor_resultado = ""
        max_longitud = 0
        for metodo, resultado in metodos_resultados.items():
            if not resultado.startswith("Error") and len(resultado) > max_longitud:
                mejor_resultado = resultado
                max_longitud = len(resultado)
        
        return {
            'texto_final': mejor_resultado,
            'metodos_intentados': metodos_resultados,
            'metodo_exitoso': max([k for k, v in metodos_resultados.items() 
                                 if not v.startswith("Error") and len(v) == max_longitud], 
                                default="ninguno")
        }
    
    def extraer_tablas_robusto(self, archivo_pdf):
        """Extrae tablas probando diferentes métodos (sin camelot)"""
        resultados_tablas = {}
        
        # tabula-py
        try:
            tablas = tabula.read_pdf(archivo_pdf, pages='all', multiple_tables=True)
            resultados_tablas['tabula'] = {
                'tablas': tablas,
                'cantidad': len(tablas),
                'exito': True
            }
        except Exception as e:
            resultados_tablas['tabula'] = {'error': str(e), 'exito': False}
        
        # pdfplumber
        try:
            with pdfplumber.open(archivo_pdf) as pdf:
                todas_tablas = []
                for page in pdf.pages:
                    tablas_pagina = page.extract_tables()
                    for tabla in tablas_pagina:
                        df = pd.DataFrame(tabla[1:], columns=tabla[0])
                        todas_tablas.append(df)
                
                resultados_tablas['pdfplumber'] = {
                    'tablas': todas_tablas,
                    'cantidad': len(todas_tablas),
                    'exito': True
                }
        except Exception as e:
            resultados_tablas['pdfplumber'] = {'error': str(e), 'exito': False}
        
        return resultados_tablas
    
    def procesar_pdf_completo(self, archivo_pdf):
        """Proceso completo adaptativo"""
        print(f"Analizando: {archivo_pdf}")
        
        info_contenido = self.detectar_tipo_contenido(archivo_pdf)
        print(f"Análisis de contenido: {info_contenido}")
        
        resultado_final = {
            'archivo': archivo_pdf,
            'analisis_contenido': info_contenido,
            'texto': None,
            'tablas': None
        }
        
        if info_contenido.get('tiene_texto') or info_contenido.get('es_escaneado'):
            print("Extrayendo texto...")
            resultado_texto = self.extraer_texto_robusto(archivo_pdf)
            resultado_final['texto'] = resultado_texto
        
        if info_contenido.get('posibles_tablas'):
            print("Extrayendo tablas...")
            resultado_tablas = self.extraer_tablas_robusto(archivo_pdf)
            resultado_final['tablas'] = resultado_tablas
        
        return resultado_final

# Ejemplo de uso
if __name__ == "__main__":
    extractor = PDFExtractorAdaptativo()
    resultado = extractor.procesar_pdf_completo("resOurces/MEGAMARKET.pdf")

CropBox missing from /Page, defaulting to MediaBox


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Analizando: resOurces/MEGAMARKET.pdf
Análisis de contenido: {'tiene_texto': True, 'tiene_imagenes': False, 'posibles_tablas': False, 'es_escaneado': False, 'calidad_texto': 'alta'}
Extrayendo texto...


In [186]:
#funcion de limpieza de columnas : 
def limpiar_columnas(df):
    df.columns = df.columns.astype(str) 
    df.columns = (
        df.columns
        .str.strip()                      
        .str.lower()                     
        .str.replace(' ', '_')        
        .str.replace(r'[^\w\s]', '', regex=True)  
    )
    return df

In [187]:
def buscar(pattern, text, group=1, flags=0):
    m = re.search(pattern, text, flags)
    return m.group(group).strip() if m else None

In [188]:
#extraer del diccionario final tablas y texto
textos = resultado.get('texto', {}).get('texto_final', '')
print(type(textos))

#Escritura .txt
with open("MEGAMARKET.txt", "w", encoding="utf-8") as f:
    f.write(textos)

<class 'str'>


In [189]:
textos

'MEGAMARKET SAS                                    Orden de Compra : 000026178\nDIR : CARRERA 58  # 9-43                                \nNIT : 811011194                 CIUDAD : MEDELLIN                 \nTEL : 6054966   \n------------------------------------------------------------------------------\nCROC SAS                                                                                                                           \nCL 69 19 36                                       \nSANTAFE DE BOGO TEL : 5448274                                                                                                      \nNIT :   830125610    COD : 050273          FAX :           COMPRADOR : 01 01 MEGAMARKET        CONTACTO:                     \n------------------------------------------------------------------------------\nDOCTO REFERENCIA            ORDEN DE COMPRA            FECHA PEDIDO     FECHA ENTREGA   TRANSPORTADOR          CCOSTO        PAG:\n000000000                   000000000000

In [190]:
texto = textos

In [191]:
datos_generales = {
    "Comprador Razon Social": buscar(r"^(MEGAMARKET SAS)", texto, flags=re.M),
    "Orden de Compra": buscar(r"Orden de Compra\s*:\s*(\d+)", texto),
    "Dir Comprador": buscar(r"DIR\s*:\s*(.*)", texto),
    "NIT Comprador": buscar(r"NIT\s*:\s*(\d+)", texto),
    "Ciudad Comprador": buscar(r"CIUDAD\s*:\s*(\w+)", texto),
    "Tel Comprador": buscar(r"TEL\s*:\s*(\d+)", texto),
    "Proveedor Razon Social": buscar(r"^(CROC SAS)", texto, flags=re.M),
    "Dir Proveedor": buscar(r"CROC SAS\s+(.*?)\n", texto, flags=re.S),
    "Tel Proveedor": buscar(r"SANTAFE DE BOGO TEL\s*:\s*(\d+)", texto),
    "NIT Proveedor": buscar(r"NIT\s*:\s*(\d+)", texto),
    "Total Cantidad": buscar(r"Total Cantidad:\s*([\d.]+)", texto),
    "Neto": buscar(r"Neto\s+([\d.,]+)", texto),
}

In [192]:
datos_generales

{'Comprador Razon Social': 'MEGAMARKET SAS',
 'Orden de Compra': '000026178',
 'Dir Comprador': 'CARRERA 58  # 9-43',
 'NIT Comprador': '811011194',
 'Ciudad Comprador': 'MEDELLIN',
 'Tel Comprador': '6054966',
 'Proveedor Razon Social': 'CROC SAS',
 'Dir Proveedor': 'CL 69 19 36',
 'Tel Proveedor': '5448274',
 'NIT Proveedor': '811011194',
 'Total Cantidad': '272.00',
 'Neto': '2,026,896.00'}

In [193]:
df_datos_generales = pd.DataFrame([datos_generales])
df_datos_generales  = limpiar_columnas(df_datos_generales )

In [194]:
df_datos_generales

,comprador_razon_social,orden_de_compra,dir_comprador,nit_comprador,ciudad_comprador,tel_comprador,proveedor_razon_social,dir_proveedor,tel_proveedor,nit_proveedor,total_cantidad,neto
0,MEGAMARKET SAS,000026178,CARRERA 58 # 9-43,811011194,MEDELLIN,6054966,CROC SAS,CL 69 19 36,5448274,811011194,272.00,"2,026,896.00"


In [195]:
texto = textos

In [196]:


# Fechas (extraídas de la línea debajo del encabezado)
fecha_linea = re.search(r"000000000\s+0+\s+(\d{2}/\d{2}/\d{4})\s+(\d{2}/\d{2}/\d{4})", texto)
fecha_pedido = fecha_linea.group(1) if fecha_linea else None
fecha_entrega = fecha_linea.group(2) if fecha_linea else None

# Diccionario de encabezado
encabezado = {
    "Orden de Compra": buscar(r"Orden de Compra\s*:\s*(\d+)", texto),
    "Dir Comprador": buscar(r"DIR\s*:\s*(.*)", texto),
    "NIT Comprador": buscar(r"NIT\s*:\s*(\d+)", texto),
    "Ciudad Comprador": buscar(r"CIUDAD\s*:\s*(\w+)", texto),
    "Tel Comprador": buscar(r"TEL\s*:\s*(\d+)", texto),
    "Proveedor": "CROC SAS",
    "Dir Proveedor": buscar(r"CROC SAS\s+(.*?)\n", texto, flags=re.S),
    "Tel Proveedor": buscar(r"SANTAFE DE BOGO TEL\s*:\s*(\d+)", texto),
    "NIT Proveedor": buscar(r"NIT\s*:\s*(\d+)", texto),
    "Fecha Pedido": fecha_pedido,
    "Fecha Entrega": fecha_entrega,
}

# Regex para productos
pattern_productos = re.compile(
    r"(\d{2})\s+(\d+)\s+(.*?)(UNIDAD)\s+([\d.]+)\s+([\d.,]+)\s+([\d.]+)\s+([\d.,]+)",
    re.DOTALL
)

matches = pattern_productos.findall(texto)

productos = []
for match in matches:
    productos.append({
        "BOD": match[0],
        "Referencia": match[1],
        "Descripción": match[2].strip(),
        "Unidad": match[3],
        "Cantidad": float(match[4]),
        "Precio Unitario": float(match[5].replace(',', '')),
        "Descuento": float(match[6]),
        "Valor Total": float(match[7].replace(',', '')),
    })

# Crear DataFrame
df_productos = pd.DataFrame(productos)



In [197]:
productos

[{'BOD': '69',
  'Referencia': '19',
  'Descripción': '36                                       \nSANTAFE DE BOGO TEL : 5448274                                                                                                      \nNIT :   830125610    COD : 050273          FAX :           COMPRADOR : 01 01 MEGAMARKET        CONTACTO:                     \n------------------------------------------------------------------------------\nDOCTO REFERENCIA            ORDEN DE COMPRA            FECHA PEDIDO     FECHA ENTREGA   TRANSPORTADOR          CCOSTO        PAG:\n000000000                   00000000000000000000        04/23/2025       04/29/2025                                           1\n------------------------------------------------------------------------------\nBOD  REFERENCIA      DESCRIPCION                              UNIDAD      CANTIDAD         PRECIO       DESCUENTO            VALOR\n01   7201005              ESPIRAL DON CHICHARRÓN X 100GR',
  'Unidad': 'UNIDAD',
  'Cantid

In [198]:
encabezado

{'Orden de Compra': '000026178',
 'Dir Comprador': 'CARRERA 58  # 9-43',
 'NIT Comprador': '811011194',
 'Ciudad Comprador': 'MEDELLIN',
 'Tel Comprador': '6054966',
 'Proveedor': 'CROC SAS',
 'Dir Proveedor': 'CL 69 19 36',
 'Tel Proveedor': '5448274',
 'NIT Proveedor': '811011194',
 'Fecha Pedido': '04/23/2025',
 'Fecha Entrega': '04/29/2025'}

In [199]:
import unicodedata

In [200]:
df_encabezado_2 = pd.DataFrame([encabezado])
#clean columns
df_encabezado_2 = limpiar_columnas(df_encabezado_2)

In [201]:
df_encabezado_2

,orden_de_compra,dir_comprador,nit_comprador,ciudad_comprador,tel_comprador,proveedor,dir_proveedor,tel_proveedor,nit_proveedor,fecha_pedido,fecha_entrega
0,000026178,CARRERA 58 # 9-43,811011194,MEDELLIN,6054966,CROC SAS,CL 69 19 36,5448274,811011194,04/23/2025,04/29/2025


### Df general - Escoger Columnas -Ordenar dataframe
1. Cliente
2. Direccion
3. Destino
4. Numero de Pedido
5. Fecha de Orden
6. Fecha de Entrega
7. Id Falso (Para Cruce de Orden de compra )

In [202]:
#create columns
df_encabezado_2["CLIENTE"] = "MEGAMARKET SAS"
df_encabezado_2["DIRECCION"] = df_encabezado_2["dir_comprador"]
df_encabezado_2["DESTINO"] = df_encabezado_2["dir_comprador"]
df_encabezado_2["NUMERO_DE_PEDIDO"] = df_encabezado_2["orden_de_compra"]
df_encabezado_2["FECHA_DE_ORDEN"] = df_encabezado_2["fecha_pedido"]
df_encabezado_2["FECHA_DE_ENTREGA"] = df_encabezado_2["fecha_entrega"]

#Select columnas 
df_encabezado_2 = df_encabezado_2[[
    "CLIENTE", "DIRECCION", "DESTINO", "NUMERO_DE_PEDIDO", 
    "FECHA_DE_ORDEN", "FECHA_DE_ENTREGA"
]]

#Create id
df_encabezado_2["id"] = 1

C:\Users\Analista de  Datos\AppData\Local\Temp\ipykernel_47540\2179341462.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encabezado_2["id"] = 1


In [203]:
df_encabezado_2

,CLIENTE,DIRECCION,DESTINO,NUMERO_DE_PEDIDO,FECHA_DE_ORDEN,FECHA_DE_ENTREGA,id
0,MEGAMARKET SAS,CARRERA 58 # 9-43,CARRERA 58 # 9-43,000026178,04/23/2025,04/29/2025,1


#### Escoger Columnas Productos 
1. Numero de item
2. descripcion
3. Cantidad
4. Unidad de medida
5. Costo Unitario
6. importe
7. id falso (Unir para realizar insersion de datos)

In [204]:
# limpieza de columnas
df_productos = limpiar_columnas(df_productos)

In [205]:
df_productos["NUMERO_DE_ITEM"] = df_productos["referencia"]
df_productos["DESCRIPCION"] = df_productos["descripción"]
df_productos["CANTIDAD"] = df_productos["cantidad"]
df_productos["UNIDAD_DE_MEDIDA"] = df_productos["unidad"]
df_productos["COSTO_UNITARIO"] = df_productos["precio_unitario"] 
df_productos["IMPORTE"] = df_productos["valor_total"] 

# create id
df_productos["id"] = 1

#Seleccionar columnas
df_productos = df_productos[[
    "id", "NUMERO_DE_ITEM", "DESCRIPCION", "CANTIDAD", 
    "UNIDAD_DE_MEDIDA", "COSTO_UNITARIO", "IMPORTE"
]]

In [206]:
df_productos

,id,NUMERO_DE_ITEM,DESCRIPCION,CANTIDAD,UNIDAD_DE_MEDIDA,COSTO_UNITARIO,IMPORTE
0,1,19,36 \nSAN...,200.0,UNIDAD,6641.76,1328352.0
1,1,7201006,ESPIRAL DON CHICHARRON NAT X 6 UDS X 21G,72.0,UNIDAD,9702.00,698544.0


#### Join

In [209]:
df_join = pd.merge(df_encabezado_2, df_productos, on="id", how="inner")

#drop duplicates
df_join = df_join.drop_duplicates()

#Reordenar Columnas 
df_join = df_join[['id', 'CLIENTE', 'DIRECCION', 'DESTINO', 'NUMERO_DE_PEDIDO', 'FECHA_DE_ORDEN', 'FECHA_DE_ENTREGA',
                   'NUMERO_DE_ITEM', 'DESCRIPCION', 'CANTIDAD', 'UNIDAD_DE_MEDIDA', 'COSTO_UNITARIO', 'IMPORTE']]

In [210]:
df_join


,id,CLIENTE,DIRECCION,DESTINO,NUMERO_DE_PEDIDO,FECHA_DE_ORDEN,FECHA_DE_ENTREGA,NUMERO_DE_ITEM,DESCRIPCION,CANTIDAD,UNIDAD_DE_MEDIDA,COSTO_UNITARIO,IMPORTE
0,1,MEGAMARKET SAS,CARRERA 58 # 9-43,CARRERA 58 # 9-43,000026178,04/23/2025,04/29/2025,19,36 \nSAN...,200.0,UNIDAD,6641.76,1328352.0
1,1,MEGAMARKET SAS,CARRERA 58 # 9-43,CARRERA 58 # 9-43,000026178,04/23/2025,04/29/2025,7201006,ESPIRAL DON CHICHARRON NAT X 6 UDS X 21G,72.0,UNIDAD,9702.00,698544.0


In [ ]:
#Homologar Productos con referencias , Destino Medellin